# A python node analysis jupyter notebook

**Synopsis:** This notebook will read in a spice like circuit netlist file and compute the node equations. The code follows Erik Cheever's Analysis of  Resistive Circuits [page](http://www.swarthmore.edu/NatSci/echeeve1/Ref/mna/MNA1.html) to generate modified nodal equations. I somewhat followed his matlab file.

**Description:**

```
Date started: April 17, 2017
file name: node analysis.ipynb
Requires: Python version 3 or higher and a jupyter notebook
Author: Tony

Revision History
7/1/2015: Ver 1 - coding started, derived from network.c code
8/18/2017
changed approach, now implementing a modified nodal analysis
8/19/2017
Wrote some code to generate symbolic matrices, works ok,
so heading down the sympy path. Basic debugging finished,
but still need to verify some circuits using Ls and Cs.
8/30/2017
Started to add code for op amps
9/1/2017
Code added to process op amps
9/3/2017
Added code to remove spice directives.
Fixed orientation of current sources in I matrix.
N2 is the arrow end of the current source.
9/4/2017

```

In [75]:
import os
from sympy import *
import numpy as np
import pandas as pd
init_printing()

In [76]:
# initialize some variables, count the types of elements
num_rlc = 0 # number of passive elements
num_ind = 0 # number of inductors
num_v = 0    # number of independent voltage sources
num_i = 0    # number of independent current sources
num_opamps = 0   # number of op amps
num_vcvs = 0     # number of controlled sources of various types
num_vccs = 0
num_cccs = 0
num_ccvs = 0
num_cpld_ind = 0 # number of coupled inductors

##### open file and preprocess it, file name extenstion is defaulted to .net
- remove blank lines and comments
- convert first letter of element name to upper case
- removes extra spaces between entries
- count number of entries on each line, make sure the count is correct

In [77]:
fn = 'RCL circuit'
fd1 = open(fn+'.net','r')
content = fd1.readlines()
content = [x.strip() for x in content]  #remove leading and trailing white space
# remove empty lines
while '' in content:
    content.pop(content.index(''))

# remove comment lines, these start with a asterisk *
content = [n for n in content if not n.startswith('*')]
# remove other comment lines, these start with a semicolon ;
content = [n for n in content if not n.startswith(';')]
# remove spice directives, these start with a period, .
content = [n for n in content if not n.startswith('.')]
# converts 1st letter to upper case
#content = [x.upper() for x in content] <- this converts all to upper case
content = [x.capitalize() for x in content]
# removes extra spaces between entries
content = [' '.join(x.split()) for x in content]

In [78]:
line_cnt = len(content) # number of lines in the netlist
branch_cnt = 0  # number of btanches in the netlist
# check number of entries on each line
for i in range(line_cnt):
    x = content[i][0]
    tk_cnt = len(content[i].split())

    if (x == 'R') or (x == 'L') or (x == 'C'):
        if tk_cnt != 4:
            print("branch {:d} not formatted correctly, {:s}".format(i,content[i]))
            print("had {:d} items and should only be 4".format(tk_cnt))
        num_rlc += 1
        branch_cnt += 1
        if x == 'L':
            num_ind += 1
    elif x == 'V':
        if tk_cnt != 4:
            print("branch {:d} not formatted correctly, {:s}".format(i,content[i]))
            print("had {:d} items and should only be 4".format(tk_cnt))
        num_v += 1
        branch_cnt += 1
    elif x == 'I':
        if tk_cnt != 4:
            print("branch {:d} not formatted correctly, {:s}".format(i,content[i]))
            print("had {:d} items and should only be 4".format(tk_cnt))
        num_i += 1
        branch_cnt += 1
    elif x == 'O':
        if tk_cnt != 4:
            print("branch {:d} not formatted correctly, {:s}".format(i,content[i]))
            print("had {:d} items and should only be 4".format(tk_cnt))
        num_opamps += 1
    elif x == 'E':
        if (tk_cnt != 6):
            print("branch {:d} not formatted correctly, {}".format(i,content[i]))
            print("had {:d} items and should only be 6".format(tk_cnt))
        num_vcvs += 1
        branch_cnt += 1
    elif x == 'G':
        if (tk_cnt != 6):
            print("branch {:d} not formatted correctly, {}".format(i,content[i]))
            print("had {:d} items and should only be 6".format(tk_cnt))
        num_vccs += 1
        branch_cnt += 1
    elif x == 'F':
        if (tk_cnt != 5):
            print("branch {:d} not formatted correctly, {}".format(i,content[i]))
            print("had {:d} items and should only be 5".format(tk_cnt))
        num_cccs += 1
        branch_cnt += 1
    elif x == 'H':
        if (tk_cnt != 5):
            print("branch {:d} not formatted correctly, {}".format(i,content[i]))
            print("had {:d} items and should only be 5".format(tk_cnt))
        num_ccvs += 1
        branch_cnt += 1
    elif x == 'K':
        if (tk_cnt != 4):
            print("branch {:d} not formatted correctly, {}".format(i,content[i]))
            print("had {:d} items and should only be 4".format(tk_cnt))
        num_cpld_ind += 1
    else:
        print("unknown element type in branch {:d}, {}".format(i,content[i]))

##### parser
- puts branch elements into structure
- counts number of nodes

data frame lables:
- count: data frame index
- element: type of element
- p node: positive node
- n node: negitive node, for a current source, the arrow terminal
- cp node: controlling positive node of branch
- cn node: controlling negitive node of branch
- Vout: opamp output node
- value: value of element or voltage
- Vname: voltage source through which the controlling current flows. Need to add a zero volt voltage source to the controlling branch.
- Lname1: name of coupled inductor 1
- Lname2: name of coupled inductor 2

```
temp code delete later
count = []        # data frame index
element = []      # type of element
p_node = []       # positive node
n_node = []       # neg node, for a current source, the arrow terminal
cp_node = []      # controlling positive node of branch
cn_node = []      # controlling negitive node of branch
Vout = []         # op amp output node
value = []        # value of element or voltage
v_name = []       # voltage source through which the controlling current flows
l_name1 = []      # name of coupled inductor 1
l_name2 = []      # name of coupled inductor 2
df = pd.DataFrame(index=count, columns=['element','p node','n node','cp node','cn node',
    'Vout','value','Vname','Lname1','Lname2'])
```

In [79]:
# build the pandas data frame
df = pd.DataFrame(columns=['element','p node','n node','cp node','cn node',
    'Vout','value','Vname','Lname1','Lname2'])

##### functions to load branch elements into data frame

In [80]:
# loads voltage or current sources into branch structure
def indep_source(line_nu):
    tk = content[line_nu].split()
    df.loc[line_nu,'element'] = tk[0]
    df.loc[line_nu,'p node'] = int(tk[1])
    df.loc[line_nu,'n node'] = int(tk[2])
    df.loc[line_nu,'value'] = float(tk[3])

# loads passive elements into branch structure
def rlc_element(line_nu):
    tk = content[line_nu].split()
    df.loc[line_nu,'element'] = tk[0]
    df.loc[line_nu,'p node'] = int(tk[1])
    df.loc[line_nu,'n node'] = int(tk[2])
    df.loc[line_nu,'value'] = float(tk[3])

'''
loads multi-terminal elements into branch structure
Types:
E - VCVS
G - VCCS
F - CCCS
H - CCVS
K - Coupled inductors
O - Op Amps
'''
def opamp_sub_network(line_nu):
    tk = content[line_nu].split()
    df.loc[line_nu,'element'] = tk[0]
    df.loc[line_nu,'p node'] = int(tk[1])
    df.loc[line_nu,'n node'] = int(tk[2])
    df.loc[line_nu,'Vout'] = int(tk[3])

def vccs_sub_network(line_nu):
    tk = content[line_nu].split()
    df.loc[line_nu,'element'] = tk[0]
    df.loc[line_nu,'p node'] = int(tk[1])
    df.loc[line_nu,'n node'] = int(tk[2])
    df.loc[line_nu,'cp node'] = int(tk[3])
    df.loc[line_nu,'cn node'] = int(tk[4])
    df.loc[line_nu,'value'] = float(tk[5])

def vcvs_sub_network(line_nu):
    tk = content[line_nu].split()
    df.loc[line_nu,'element'] = tk[0]
    df.loc[line_nu,'p node'] = int(tk[1])
    df.loc[line_nu,'n node'] = int(tk[2])
    df.loc[line_nu,'cp node'] = int(tk[3])
    df.loc[line_nu,'cn node'] = int(tk[4])
    df.loc[line_nu,'value'] = float(tk[5])

def cccs_sub_network(line_nu):
    tk = content[line_nu].split()
    df.loc[line_nu,'element'] = tk[0]
    df.loc[line_nu,'p node'] = int(tk[1])
    df.loc[line_nu,'n node'] = int(tk[2])
    df.loc[line_nu,'Vname'] = tk[3].capitalize()
    df.loc[line_nu,'value'] = float(tk[4])

def ccvs_sub_network(line_nu):
    tk = content[line_nu].split()
    df.loc[line_nu,'element'] = tk[0]
    df.loc[line_nu,'p node'] = int(tk[1])
    df.loc[line_nu,'n node'] = int(tk[2])
    df.loc[line_nu,'Vname'] = tk[3].capitalize()
    df.loc[line_nu,'value'] = float(tk[4])

def cpld_ind_sub_network(line_nu):
    tk = content[line_nu].split()
    df.loc[line_nu,'element'] = tk[0]
    df.loc[line_nu,'Lname1'] = tk[1].capitalize()
    df.loc[line_nu,'Lname2'] = tk[2].capitalize()
    df.loc[line_nu,'value'] = float(tk[3])

In [81]:
# function to scan df and get largest node number
def count_nodes():
    # need to check that nodes are consecutive
    # fill array with node numbers
    p = np.zeros(line_cnt+1)
    for i in range(line_cnt-1):
        p[df['p node'][i]] = df['p node'][i]
        p[df['n node'][i]] = df['n node'][i]

    # find the largest node number
    if df['n node'].max() > df['p node'].max():
        largest = df['n node'].max()
    else:
        largest =  df['p node'].max()

    largest = int(largest)
    # check for unfilled elements, skip node 0
    for i in range(1,largest):
        if p[i] == 0:
            print('nodes not in continuous order, node {:.0f} is missing'.format(p[i-1]+1))

    return largest

In [82]:
# load branch info into data frame
for i in range(line_cnt):
    x = content[i][0]

    if (x == 'R') or (x == 'L') or (x == 'C'):
        rlc_element(i)
    elif (x == 'V') or (x == 'I'):
        indep_source(i)
    elif x == 'O':
        opamp_sub_network(i)
    elif x == 'E':
        vcvs_sub_network(i)
    elif x == 'G':
        vccs_sub_network(i)
    elif x == 'F':
        cccs_sub_network(i)
    elif x == 'H':
        ccvs_sub_network(i)
    elif x == 'K':
        cpld_ind_sub_network(i)
    else:
        print("unknown element type in branch {:d}, {}".format(i,content[i]))

# count number of nodes
num_nodes = count_nodes()

In [83]:
# print a report
print('Net list report')
print('number of lines in netlist: {:d}'.format(line_cnt))
print('number of branches: {:d}'.format(branch_cnt))
print('number of nodes: {:d}'.format(num_nodes))
print('number of passive components: {:d}'.format(num_rlc))
print('number of inductors: {:d}'.format(num_ind))
print('number of independent voltage sources: {:d}'.format(num_v))
print('number of independent current sources: {:d}'.format(num_i))
print('number of op amps: {:d}'.format(num_opamps))

# not implemented yet
print('\nNot implemented yet')
print('number of E - VCVS: {:d}'.format(num_vcvs))
print('number of G - VCCS: {:d}'.format(num_vccs))
print('number of F - CCCS: {:d}'.format(num_cccs))
print('number of F - CCCS: {:d}'.format(num_ccvs))
print('number of K - Coupled inductors: {:d}'.format(num_cpld_ind))

Net list report
number of lines in netlist: 6
number of branches: 6
number of nodes: 3
number of passive components: 5
number of inductors: 1
number of independent voltage sources: 1
number of independent current sources: 0
number of op amps: 0

Not implemented yet
number of E - VCVS: 0
number of G - VCCS: 0
number of F - CCCS: 0
number of F - CCCS: 0
number of K - Coupled inductors: 0


In [84]:
content

['Vin 3 0 1',
 'R2 3 2 1000',
 'R1 1 0 1000',
 'C1 1 0 1e-6',
 'C2 2 1 10e-6',
 'L1 1 0 0.001']

In [85]:
df

,element,p node,n node,cp node,cn node,Vout,value,Vname,Lname1,Lname2
0,Vin,3,0,NaN,NaN,NaN,1,NaN,NaN,NaN
1,R2,3,2,NaN,NaN,NaN,1000,NaN,NaN,NaN
2,R1,1,0,NaN,NaN,NaN,1000,NaN,NaN,NaN
3,C1,1,0,NaN,NaN,NaN,1e-06,NaN,NaN,NaN
4,C2,2,1,NaN,NaN,NaN,1e-05,NaN,NaN,NaN
5,L1,1,0,NaN,NaN,NaN,0.001,NaN,NaN,NaN


In [86]:
# store the data frame as a pickle file
# df.to_pickle(fn+'.pkl')  # <- uncomment if needed

In [87]:
# initialize some symbolic matrix with zeros
# A is formed by [[G, C] [B, D]]
# Z = [I,E]
# X = [V, J]
V = zeros(num_nodes,1)
I = zeros(num_nodes,1)
G = zeros(num_nodes,num_nodes)
s = Symbol('s')  # the Laplace variable

# count the number of element types that affect the size of the B, C, D, E and J arrays
k = num_v+num_opamps+num_vcvs+num_ccvs+num_ind
if (num_v+num_opamps) != 0:
    B = zeros(num_nodes,num_v+num_opamps)
    C = zeros(num_v+num_opamps,num_nodes)
    D = zeros(num_v+num_opamps,num_v+num_opamps)
    E = zeros(num_v+num_opamps,1)
    J = zeros(num_v+num_opamps,1)

##### G matrix                  <span style="color:red">\----need to check on inductor treatment, doesn't verify with LTspice testing, inductor stamp affects the B,C and D arrays</span>
The G matrix is n by n and is determined by the interconnections between the passive circuit elements (RLC's).  The G matrix is an nxn matrix formed in two steps:
1. Each element in the diagonal matrix is equal to the sum of the conductance (one over the resistance) of each element connected to the corresponding node.  So the first diagonal element is the sum of conductances connected to node 1, the second diagonal element is the sum of conductances connected to node 2, and so on.
2. The off diagonal elements are the negative conductance of the element connected to the pair of corresponding node.  Therefore a resistor between nodes 1 and 2 goes into the G matrix at location (1,2) and locations (2,1).

In [88]:
# G matrix
for i in range(branch_cnt):  # don't use branch count use # of rows in data frame
    n1 = df.loc[i,'p node']
    n2 = df.loc[i,'n node']
    # process all the passive elements, save conductance to temp value
    x = df.loc[i,'element'][0]   #get 1st letter of element name
    if x == 'R':
        g = 1/sympify(df.loc[i,'element'])
    if x == 'L':
        g = 1/s/sympify(df.loc[i,'element'])  # this matches Eric's code
    if x == 'C':
        g = s*sympify(df.loc[i,'element'])

    if (x == 'R') or (x == 'L') or (x == 'C'):
        # If neither side of the element is connected to ground
        # then subtract it from appropriate location in matrix.
        if (n1 != 0) and (n2 != 0):
            G[n1-1,n2-1] += -g
            G[n2-1,n1-1] += -g

        # If node 1 is connected to ground, add element to diagonal of matrix
        if n1 != 0:
            G[n1-1,n1-1] += g

        # same for for node 2
        if n2 != 0:
            G[n2-1,n2-1] += g

G  # display the G matrix

##### I matrix
The I matrix is an n by 1 matrix with each element of the matrix corresponding to a particular node.  The value of each element of I is determined by the sum of current sources into the corresponding node.  If there are no current sources connected to the node, the value is zero.

In [89]:
# generate the I matrix, current sources have N2 = arrow end
for i in range(branch_cnt):
    n1 = df.loc[i,'p node']
    n2 = df.loc[i,'n node']
    # process all the passive elements, save conductance to temp value
    x = df.loc[i,'element'][0]   #get 1st letter of element name
    if x == 'I':
        g = sympify(df.loc[i,'element'])
        # sum the current into each node
        if n1 != 0:
            I[n1-1] -= g
        if n2 != 0:
            I[n2-1] += g

I  # display the I matrix

##### V matrix
The V matrixis an nx1 matrix formed of the node voltages.  Each element in V corresponds to the voltage at the equivalent node in the circuit

In [90]:
# generate the V matrix
for i in range(num_nodes):
    V[i] = sympify('v{:d}'.format(i+1))

V  # display the V matrix

##### B Matrix
Rules for making the B matrix
The B matrix is an n by m matrix with only 0, 1 and -1 elements (except for controlled sources).  Each location in the matrix corresponds to a particular voltage source (first dimension) or a node (second dimension).  If the positive terminal of the ith voltage source is connected to node k, then the element (i,k) in the B matrix is a 1.  If the negative terminal of the ith voltage source is connected to node k, then the element (i,k) in the B matrix is a -1.  Otherwise, elements of the B matrix are zero.

In [91]:
# generate the B Matrix
# loop through all the branches and process independent voltage sources
sn = 0   # count source number
for i in range(branch_cnt):
    n1 = df.loc[i,'p node']
    n2 = df.loc[i,'n node']
    # process all the independent voltage sources
    x = df.loc[i,'element'][0]   #get 1st letter of element name
    if x == 'V':
        if num_v+num_opamps > 1:
            if n1 != 0:
                B[n1-1,sn] = 1
            if n2 != 0:
                B[n2-1,sn] = -1
            sn += 1   #increment source count
        else:
            if n1 != 0:
                B[n1-1] = 1
            if n2 != 0:
                B[n2-1] = -1

# loop through all the branches and process op amps
oan = 0   # running count of op amp number
for i in range(branch_cnt):
    n_vout = df.loc[i,'Vout'] # node connected to op amp output
    # look for branches with op amps and process
    x = df.loc[i,'element'][0]   # get 1st letter of element name
    if x == 'O':
        B[n_vout-1,oan+num_v] = 1
        oan += 1   # increment op amp count

B   # display the B matrix

##### J matrix
The is an m by 1 matrix, with one entry for the current through each voltage source.

In [92]:
# The J matrix is an mx1 matrix, with one entry for the current through each voltage source.
sn = 0   # count source number
oan = 0   #count op amp number
for i in range(branch_cnt):
    # process all the passive elements
    x = df.loc[i,'element'][0]   #get 1st letter of element name
    if x == 'V':
        J[sn] = sympify('I_{:s}'.format(df.loc[i,'element']))
        sn += 1
    if x == 'O':  # this needs to be checked <---- needs debugging
        J[oan+num_v] = sympify('I_{:s}'.format(df.loc[i,'element']))
        oan += 1

J  # diplay the J matrix

##### C matrix
The C matrix is an m by n matrix with only 0, 1 and -1 elements (except for controlled sources).  Each location in the matrix corresponds to a particular node (first dimension) or voltage source (second dimension).  If the positive terminal of the ith voltage source is connected to node k, then the element (k,i) in the C matrix is a 1.  If the negative terminal of the ith voltage source is connected to node k, then the element (k,i) in the C matrix is a -1.  Otherwise, elements of the C matrix are zero.

In [93]:
# generate the C matrix
sn = 0   # count source number
for i in range(branch_cnt):
    n1 = df.loc[i,'p node']
    n2 = df.loc[i,'n node']
    # process all the independent voltage sources
    x = df.loc[i,'element'][0]   #get 1st letter of element name
    if x == 'V':
        if num_v+num_opamps > 1:
            if n1 != 0:
                C[sn,n1-1] = 1
            if n2 != 0:
                C[sn,n2-1] = -1
            sn += 1   #increment source count
        else:
            if n1 != 0:
                C[n1-1] = 1
            if n2 != 0:
                C[n2-1] = -1

# loop through all the branches and process op amps
oan = 0   # running count of op amp number
for i in range(branch_cnt):
    n1 = df.loc[i,'p node']
    n2 = df.loc[i,'n node']
    n_vout = df.loc[i,'Vout'] # node connected to op amp output
    # look for branches with op amps and process
    x = df.loc[i,'element'][0]   # get 1st letter of element name
    if x == 'O':
        if n1 != 0:
            C[oan+num_v,n1-1] = 1
        if n2 != 0:
            C[oan+num_v,n2-1] = -1
        oan += 1  # increment op amp number

C   # display the C matrix

##### D matrix
The D matrix is an mxm matrix that is composed entirely of zeros.  (It can be non-zero if controlled sources are considered.)

In [94]:
# display the The D matrix
D

##### E matrix
The E matrix is mx1 and holds the values of the independent voltage sources.

In [95]:
# generate the E matrix
sn = 0   # count source number
for i in range(branch_cnt):
    # process all the passive elements
    x = df.loc[i,'element'][0]   #get 1st letter of element name
    if x == 'V':
        E[sn] = sympify(df.loc[i,'element'])
        sn += 1

E   # display the E matrix

##### Z matrix
The Z matrix holds the independent voltage and current sources and is the combination of 2 smaller matrices I and E.  The Z matrix is (m+n) by 1, n is the number of nodes, and m is the number of independent voltage sources.  The I matrix is n by 1 and contains the sum of the currents through the passive elements into the corresponding node (either zero, or the sum of independent current sources). The E matrix is m by 1 and holds the values of the independent voltage sources.

In [96]:
Z = I[:] + E[:]
Z  # display the Z matrix

##### X matrix
The X matrix is an (n+m) by 1 vector that holds the unknown quantities (node voltages and the currents through the independent voltage sources). The top n elements are the n node voltages. The bottom m elements represent the currents through the m independent voltage sources in the circuit. The V matrix is n by 1 and holds the unknown voltages.  The J matrix is m by 1 and holds the unknown currents through the voltage sources

In [97]:
X = V[:] + J[:]
X  # display the X matrix

##### A matrix
The A matrix is (m+n) by (m+n) and will be developed as the combination of 4 smaller matrices, G, B, C, and D.

In [98]:
n = num_nodes
m = num_v+num_opamps
A = zeros(m+n,m+n)
for i in range(n):
    for j in range(n):
        A[i,j] = G[i,j]

if num_v+num_opamps > 1:
    for i in range(n):
        for j in range(m):
            A[i,n+j] = B[i,j]
            A[n+j,i] = C[j,i]
else:
    for i in range(n):
        A[i,n] = B[i]
        A[n,i] = C[i]

A  # display the A matrix

In [99]:
# generate the circuit equations
n = num_nodes
m = num_v+num_opamps
eq_temp = 0  # temporary equation used to build up the equation
equ = zeros(m+n,1)  #initialize the array to hold the equations
for i in range(n+m):
    for j in range(n+m):
        eq_temp += A[i,j]*X[j]
    equ[i] = Eq(eq_temp,Z[i])
    eq_temp = 0

equ   # display the equations

Use the str() function to convert sympy equations to strings.  These strings can be copid to a new notebook.

In [100]:
str(equ)

'Matrix([[Eq(-C2*s*v2 + v1*(C1*s + C2*s + 1/R1 + 1/(L1*s)), 0)], [Eq(-C2*s*v1 + v2*(C2*s + 1/R2) - v3/R2, 0)], [Eq(I_Vin - v2/R2 + v3/R2, 0)], [Eq(v3, Vin)]])'

In [101]:
str(equ.free_symbols)

'{Vin, R1, L1, v2, R2, C2, v1, s, v3, I_Vin, C1}'

In [102]:
str(X)

'[v1, v2, v3, I_Vin]'

In [103]:
df

,element,p node,n node,cp node,cn node,Vout,value,Vname,Lname1,Lname2
0,Vin,3,0,NaN,NaN,NaN,1,NaN,NaN,NaN
1,R2,3,2,NaN,NaN,NaN,1000,NaN,NaN,NaN
2,R1,1,0,NaN,NaN,NaN,1000,NaN,NaN,NaN
3,C1,1,0,NaN,NaN,NaN,1e-06,NaN,NaN,NaN
4,C2,2,1,NaN,NaN,NaN,1e-05,NaN,NaN,NaN
5,L1,1,0,NaN,NaN,NaN,0.001,NaN,NaN,NaN
